In [1]:
import numpy as np
import importlib
import util as uti
import matplotlib.pyplot as plt
importlib.reload(uti)

# import math lib
from math import pi

# import Qiskit
from qiskit import Aer, IBMQ, execute, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library import GroverOperator
from qiskit_machine_learning.circuit.library import RawFeatureVector

# import basic plot tools
#import qiskit.visualization as vis
from qiskit.visualization import plot_histogram
%matplotlib inline

/home/kinga/.local/lib/python3.6/site-packages/qiskit/__init__.py:66: DeprecationWarning: Using Qiskit with Python 3.6 is deprecated as of qiskit-terra 0.17.0. Support for running Qiskit with Python 3.6 will be removed in qiskit-terra 0.20.0.
  DeprecationWarning,


In [6]:
def recur_list_test(ll, s): # list argument passed as reference copy
    if s < 7:
        recur_list_test(ll, s+1)
        ll.append(s)
    ll = [1] # has no effect (new assignment to reference copy)

In [7]:
ol = []
recur_list_test(ol, 2)

In [8]:
ol

[6, 5, 4, 3, 2]

In [34]:
list(range(3,7))

[3, 4, 5, 6]

In [14]:
def recursive_compute_beta(input_vector, betas):
    """
    recursive rotation computetation
    source: https://www.nature.com/articles/s41598-021-85474-1.pdf
    """
    if len(input_vector) > 1:
        new_x = []
        beta = []
        for k in range(0, len(input_vector), 2):
            norm = np.sqrt(input_vector[k] ** 2 + input_vector[k + 1] ** 2)
            new_x.append(norm)
            if norm == 0:
                beta.append(0)
            else:
                if input_vector[k] < 0:
                    beta.append(2 * np.pi - 2 * np.arcsin(input_vector[k + 1] / norm))
                else:
                    beta.append(2 * np.arcsin(input_vector[k + 1] / norm))
        recursive_compute_beta(new_x, betas)
        betas.append(beta)

In [15]:
def index(k, circuit, control_qubits, numberof_controls):
    binary_index = '{:0{}b}'.format(k, numberof_controls)
    for j, qbit in enumerate(control_qubits):
        if binary_index[j] == '1':
            circuit.x(qbit)

In [24]:
def generate_circuit(betas, qcircuit, quantum_input, num_qubits):
    numberof_controls = 0  # number of controls
    control_bits = []
    for angles in betas:
        if numberof_controls == 0:
            qcircuit.ry(angles[0], quantum_input[num_qubits-1])
            numberof_controls += 1
            control_bits.append(quantum_input[num_qubits-1])
        else:
            for k, angle in enumerate(reversed(angles)):
                index(k, qcircuit, control_bits, numberof_controls)

                qcircuit.mcry(angle, control_bits, quantum_input[num_qubits - 1 - numberof_controls],
                              None, mode='noancilla')

                index(k, qcircuit, control_bits, numberof_controls)
                
            control_bits.append(quantum_input[num_qubits - 1 - numberof_controls])
            numberof_controls += 1

In [25]:
def amplitude_encoding(input_vector):
    """
    load real vector x to the amplitude of a quantum state
    source: https://www.nature.com/articles/s41598-021-85474-1.pdf
    """
    num_qubits = int(np.log2(len(input_vector)))
    quantum_data = QuantumRegister(num_qubits)
    qcircuit = QuantumCircuit(quantum_data)
    newx = np.copy(input_vector)
    betas = []
    recursive_compute_beta(newx, betas)
    generate_circuit(betas, qcircuit, quantum_data, num_qubits)
    return qcircuit

In [28]:
v1 = np.array([0.2, 1.5, 0.7, 0.99])
qc_v1 = amplitude_encoding(v1)

In [29]:
qc_v1.draw()

┌─────────────────┐     ┌─────────────────┐     
q4_0: ─────────────┤ U(1.9107,0,0,0) ├─────┤ U(2.8765,0,0,0) ├─────
      ┌───────────┐└────────┬────────┘┌───┐└────────┬────────┘┌───┐
q4_1: ┤ Ry(1.351) ├─────────■─────────┤ X ├─────────■─────────┤ X ├
      └───────────┘                   └───┘                   └───┘

In [30]:
v2 = np.array([3.5, 0.5, 0.02, 6.7])
qc_v2 = amplitude_encoding(v2)

In [31]:
qc_v2.draw()

┌─────────────────┐     ┌──────────────────┐     
q5_0: ──────────────┤ U(3.1356,0,0,0) ├─────┤ U(0.28379,0,0,0) ├─────
      ┌────────────┐└────────┬────────┘┌───┐└────────┬─────────┘┌───┐
q5_1: ┤ Ry(2.1705) ├─────────■─────────┤ X ├─────────■──────────┤ X ├
      └────────────┘                   └───┘                    └───┘

In [36]:
# basic swap test directly on quantum states that are rotated against each other
def swap_test(v1, v2):
    
    backend = Aer.get_backend('qasm_simulator')
    # create circuit with n qubits and one classical bit for measurement
    # qubits: first: ancilla, second: input vector, third: cluster center
    #(for latent space in R^2: 1 (ancilla) + 1 (x1,x2 coords of input) + 1 (x1,x2 coords of cluster))
    n = int(np.log2(len(v1)))
    qc = QuantumCircuit(n*2+1, 1, name="swap_test")

    # control qubit default 0
    # append first vector
    qc.append(v1, range(1,n+1))
    # append second vector
    qc.append(v2, range(n+1,n*2+1))
    # third qubit init to phi
    qc.barrier()
    # apply hadamard to control
    qc.h(0)
    # swap pairwise qubits controlled on ancilla
    for i in range(n):
        swap_test_circuit.cswap(0,i+1,i+n)
    # apply second hadamard
    qc.h(0)
    # measure control qubit
    qc.measure(0,0)
    qc.draw()
    
    shots = 1024
    qobj = assemble(qc, backend)
    results = backend.run(qobj).result()
    answer = results.get_counts()

    return qc, answer

In [38]:
qc, answer = swap_test(qc_v1, qc_v2)

NameError: name 'num_qubits' is not defined